# Coursera Capstone Project

# Part I: Segmenting and Clustering Neighborhoods in Toronto
Explore and Cluster the neighborhoods in Toronto

In [1]:
import pandas as pd

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df=pd.read_html(url)

In [8]:
df_toronto=df[0]
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [15]:
#Check for any 'Not assigned'  Neighnourhood but having a valid name in Borough
df_toronto[(df_toronto['Neighbourhood']=='Not assigned') & (df_toronto['Borough']!='Not assigned')]

,Postal Code,Borough,Neighbourhood


In [19]:
toronto=df_toronto[df_toronto['Borough']!='Not assigned']
toronto.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [22]:
print('The number of rows in my dataframe is {}'.format(toronto.shape[0]))

The number of rows in my dataframe is 103
